In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [6]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [7]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.267s

OK


In [8]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [9]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [10]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [11]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.231s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [7]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, little_endian_to_int, hash256
from script import p2pkh_script, Script

from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4
passphrase = b'thiscohortrocks@satoshiwillwin.btc'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
sk = PrivateKey(secret)
pk = sk.point
# print an address from the public point of the private key with testnet=True
addr = pk.address(compressed=True, testnet=True)
print(addr)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx = bytes.fromhex('f471e342570f88af600fb352a2f3fbe2f7fc58eb41189daf90e28a569a6dcf0c')
prev_index = 1

# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_ins = []
tx_ins.append(TxIn(prev_tx, prev_index))
tx_outs = []

change_address = 'mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2'
# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee

target_amount=600
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object
tx_outs.append(TxOut(target_amount, script_pubkey))
h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
change_sats = 10 

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
tx_outs.append(TxOut(change_sats, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(tx_obj.sign_input(0, sk))
print(tx_obj.serialize().hex())


mga53D7jXBhVvT2a3vsxnAZJ2DBecqZ4NH
True
01000000010ccf6d9a568ae290af9d1841eb58fcf7e2fbf3a252b30f60af880f5742e371f4010000006a47304402200584e2f67b53d15673550722dd5b483669447b972dddba037f873cc1bb48baf802207c99493866ec72f865300e9c371815674dc61280155ca5ecbb87d1257be81aff0121023f5bfb166213d58913a46caa7edc5a6451acbcb15f2bf731e8f8f0a16df4a556ffffffff0258020000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac0a000000000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [15]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, little_endian_to_int, hash256
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
passphrase = b'thiscohortrocks@satoshiwillwin.btc'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
sk = PrivateKey(secret)
pk = sk.point
# print an address from the public point of the private key with testnet=True
addr = pk.address(compressed=True, testnet=True)
print(addr)

passphrase2 = b'stayhumbleandstacksats'
secret2 = little_endian_to_int(hash256(passphrase2))
# create a private key using your secret
sk2 = PrivateKey(secret2)
pk2 = sk2.point
# print an address from the public point of the private key with testnet=True
addr2 = pk2.address(compressed=True, testnet=True)
print(addr2)

# get the prev_tx and prev_index from the transaction where you got

prev_tx1 = bytes.fromhex('761ffe998d384c6d9629c2c70e848109ec5f350fcc6c6547724c5b3c0402e191')
prev_index1 = 1 

prev_tx2 = bytes.fromhex('345417e181b64adf6424174614c7f61e69d68c6dd2cf89bf8d642c1b5d73375d')
prev_index2 = 0

# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence
tx_ins = []
tx_ins.append(TxIn(prev_tx1, prev_index1))
tx_ins.append(TxIn(prev_tx2, prev_index2))
tx_outs = []

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee

target_amount=1200
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)

# create a transaction output for the amount and address

tx_outs.append(TxOut(target_amount, script_pubkey))

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(tx_obj.sign_input(0, sk))
print(tx_obj.sign_input(1, sk2))
print(tx_obj.serialize().hex())
# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

mga53D7jXBhVvT2a3vsxnAZJ2DBecqZ4NH
muPydW2pYk9GzfaZuWVDwaV8Yn5N4YnRr4
True
True
010000000291e102043c5b4c7247656ccc0f355fec0981840ec7c229966d4c388d99fe1f76010000006b483045022100a25709ebe39afe913de9ef5dc6c498a0a4578eb9be885fafc5f9b7f7ec6c285702207bd53e602f171a20abf3ffbe7927ba904a8b5a0417b9e13d4d7e088e6597021b0121023f5bfb166213d58913a46caa7edc5a6451acbcb15f2bf731e8f8f0a16df4a556ffffffff5d37735d1b2c648dbf89cfd26d8cd6691ef6c71446172464df4ab681e1175434000000006a47304402201f85e6f84141e8d5014b07d3db6effdd59d4125f62998c41f2520922033c20ae0220127d6d9232a075f401bdcb30d01a2c814517cb691b60b1955a7fdeca08b63dc7012103fc040e49edf18ac03f80ce6db7e3719499f6bad09cf8160937a80dc559e49e44ffffffff01b0040000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
